In [1]:
import os
import shutil

# 设置路径
data_dir = './videos/AFL_MRI'

label_dir = os.path.join(data_dir, 'label_in_png_L4L5_4class_DINO_order')
prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')


# 创建新的目标目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')
os.makedirs(output_dir, exist_ok=True)

In [2]:


# 获取 label_dir 中的文件名映射
label_mapping = {}
for label_file in os.listdir(label_dir):
    if label_file.endswith('.png'):
        file_prefix = label_file.split('_')[0]  # 获取开头的00000, 00001等
        label_mapping[file_prefix] = label_file

# 复制并重命名文件
for file in os.listdir(auto_seg_dir):
    if 'combined.png' in file:
        # 提取 frame_xx 编号
        frame_idx = file.split('_')[1]  # 获取 frame_xx 中的 xx
        frame_prefix = frame_idx.zfill(5)  # 确保前导零一致 (00000, 00001)

        # 查找匹配的 label 文件名
        if frame_prefix in label_mapping:
            new_filename = label_mapping[frame_prefix]
            src_path = os.path.join(auto_seg_dir, file)
            dst_path = os.path.join(output_dir, new_filename)

            # 复制并重命名文件
            if not os.path.exists(dst_path):
                shutil.copy(src_path, dst_path)
                print(f"Copied {file} → {new_filename}")

print(f"All files copied to {output_dir}")


All files copied to ./videos/AFL_MRI/manual_prompt_frame0/seg_mask_renamed


In [3]:
import os, glob,cv2
import SimpleITK as sitk
import numpy as np
from PIL import Image  # 用于读取 PNG 掩码

# -------------- 保存 NIfTI 文件的函数 --------------
def save_array_to_nii(mask_array, nii_template_path, nii_save_path):
    """
    将 NumPy 掩码数组保存为 NIfTI 格式，保持与模板 NIfTI 文件相同的空间信息。
    """
    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()

    # 将 NumPy 数组转换为 SimpleITK 图像对象
    nii_output = sitk.GetImageFromArray(mask_array)
    nii_output.SetSpacing(output_spacing)
    nii_output.SetDirection(output_direction)
    nii_output.SetOrigin(output_origin)

    # 保存 NIfTI 文件
    sitk.WriteImage(nii_output, nii_save_path)
    print(f"Saved: {nii_save_path}")
    
    
# -------------- 目录路径 --------------
label_nii_dir = os.path.join(data_dir, 'raw_data/AFL_label_segmentation_full_body')  # 原始 NIfTI 文件
seg_nii_dir = os.path.join(data_dir,  prompt_dir, 'seg_in_nii_L4-L5')  # 保存分割 NIfTI 的目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')  # 自动分割 PNG 掩码所在目录

# seg_nii_dir = os.path.join(data_dir, 'label_in_nii_L4-L5')  # 保存分割 NIfTI 的目录
# output_dir = os.path.join(data_dir, 'label_in_png_L4L5_4class_DINO_order')  # 自动分割 PNG 掩码所在目录

os.makedirs(seg_nii_dir, exist_ok=True)  # 创建保存路径（如果不存在）

# -------------- 遍历自动分割的掩码文件 --------------
for idx, filename in enumerate(os.listdir(output_dir)):    
    print(f"{idx}: Processing {filename}")

    # 拆分文件名，提取 `pid` 和 `slice_idx`
    parts = filename.split('_')
    pid = "_".join(parts[1:-4])  # 提取 PID，例如 "ID1A_R0"
    slice_idx = int(parts[-3].replace('slice', ''))  # 提取 Slice Index，例如 017 → 17

    # 查找匹配的 NIfTI 文件
    nii_files = glob.glob(os.path.join(label_nii_dir, pid + "*.nii.gz"))
    if not nii_files:
        print(f"Warning: No matching NIfTI file found for PID {pid}")
        continue
    label_nii_path = nii_files[0]

    # 读取 label NIfTI 作为模板
    label_sitk = sitk.ReadImage(label_nii_path)
    label_array = sitk.GetArrayFromImage(label_sitk)  # 转换为 NumPy 数组
    
    # 创建新的 mask_array（与 label_array 形状相同，全为 0）
    mask_array = np.zeros_like(label_array, dtype=np.uint8)

    # 读取 PNG 掩码文件（auto_mask）
    mask_path = os.path.join(output_dir, filename)
    auto_mask = np.array(Image.open(mask_path))

   # 获取 mask_array[slice_idx, :, :] 的目标形状
    target_shape = mask_array[slice_idx, :, :].shape  # (H, W)

    # 初始化新的 mask_mapped_resized 数组（与目标形状相同）
    mask_mapped_resized = np.zeros(target_shape, dtype=np.uint8)

    # 逐个处理每个 class，进行 resize 并合并
    for class_value in [50, 100, 150, 200]:
        binary_mask = (auto_mask == class_value).astype(np.uint8)  # 生成二值 mask
        resized_binary_mask = cv2.resize(binary_mask, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_NEAREST)  # 保持最近邻插值
        mask_mapped_resized[resized_binary_mask > 0] = {50: 1, 100: 2, 150: 3, 200: 4}[class_value]  # 赋值回正确的 class

    # 赋值到 mask_array
    if 'ID' in pid:
        mask_mapped_resized =np.rot90(mask_mapped_resized, 2) 
    mask_array[slice_idx, :, :] = mask_mapped_resized

    # 设置保存路径
    
    nii_save_path = os.path.join(seg_nii_dir, f"{pid}_L4-L5_sam2_seg_slice{slice_idx}_seg.nii.gz")
    if 'label_in_nii' in seg_nii_dir:
        nii_save_path = os.path.join(seg_nii_dir, f"{pid}_L4-L5_label_slice{slice_idx}_seg.nii.gz")
    # 保存 NIfTI 掩码文件
    if not os.path.exists(nii_save_path):
        save_array_to_nii(mask_array, label_nii_path, nii_save_path)

        print(f"PID: {pid}, Slice Index: {slice_idx}")
        print(f"Label NIfTI Path: {label_nii_path}")
        print(f"Saved Segmentation: {nii_save_path}\n\n")


0: Processing 00036_Jake_145_slice_015_imgseg_.png
1: Processing 00033_Jake_144_slice_016_imgseg_.png
2: Processing 00023_GL_116_slice_016_imgseg_.png
3: Processing 00021_Jake_148_slice_013_imgseg_.png
4: Processing 00039_Jake_106_slice_015_imgseg_.png
5: Processing 00022_Jake_136_slice_015_imgseg_.png
6: Processing 00035_Jake_138_slice_019_imgseg_.png
7: Processing 00000_Jake_105_slice_019_imgseg_.png
8: Processing 00026_GL_110_slice_022_imgseg_.png
9: Processing 00020_GL_101_slice_013_imgseg_.png
10: Processing 00034_N_Melb_115_slice_015_imgseg_.png
11: Processing 00016_GL_105_slice_014_imgseg_.png
12: Processing 00019_GL_109_slice_018_imgseg_.png
13: Processing 00014_GL_108_slice_016_imgseg_.png
14: Processing 00027_Jake_142_slice_015_imgseg_.png
15: Processing 00042_GL_112_slice_017_imgseg_.png
16: Processing 00006_Jake_137_slice_017_imgseg_.png
17: Processing 00012_GL_102_slice_015_imgseg_.png
18: Processing 00025_N_Melb_101_slice_015_imgseg_.png
19: Processing 00028_GL_104_slice_

In [4]:
import os
import glob
import SimpleITK as sitk
import numpy as np
import pandas as pd


# 目录设置
auto_seg_dir = os.path.join(data_dir, prompt_dir, 'seg_in_nii_L4-L5')    # 自动分割结果所在目录
manual_seg_dir = os.path.join(data_dir, 'label_in_nii_L4-L5')             # 手动分割结果所在目录

# 创建 DataFrame 用于存储结果
df_results = pd.DataFrame(columns=["PID", "Class", "Auto Area", "Manual Area"])

# 遍历自动分割结果文件
for idx, auto_filename in enumerate(os.listdir(auto_seg_dir)):
    print(f"{idx}: Processing {auto_filename}")

    # 提取 PID，假设文件名格式类似 "ID1A_R0_XXX.nii.gz"，PID 为前两段 "ID1A_R0"
    parts = auto_filename.split('_')
    pid = "_".join(parts[0:2])

    # 自动分割文件读取
    auto_path = os.path.join(auto_seg_dir, auto_filename)
    auto_img = sitk.ReadImage(auto_path)
    auto_array = sitk.GetArrayFromImage(auto_img)
    auto_spacing = auto_img.GetSpacing()  # 例如 (spacing_x, spacing_y, spacing_z)

    # 查找对应的手动分割文件（根据 PID 匹配）
    manual_files = glob.glob(os.path.join(manual_seg_dir, pid + "*.nii.gz"))
    if not manual_files:
        print(f"Warning: Manual segmentation file not found for {pid}")
        continue
    manual_path = manual_files[0]
    manual_img = sitk.ReadImage(manual_path)
    manual_array = sitk.GetArrayFromImage(manual_img)
    manual_spacing = manual_img.GetSpacing()

    # 对于每个类别（例如 1,2,3,4）计算面积（单位：原面积除以 100，并保留 3 位小数）
    for class_value in [1, 2, 3, 4]:
        # 自动分割中该类别的像素数及面积
        auto_mask = (auto_array == class_value)
        auto_pixel_count = np.sum(auto_mask)
        auto_area = auto_pixel_count * auto_spacing[0] * auto_spacing[1]
        auto_area = round(auto_area / 100, 3)

        # 手动分割中该类别的像素数及面积
        manual_mask = (manual_array == class_value)
        manual_pixel_count = np.sum(manual_mask)
        manual_area = manual_pixel_count * manual_spacing[0] * manual_spacing[1]
        manual_area = round(manual_area / 100, 3)

        # 将结果追加到 DataFrame 中
        df_results = df_results._append({
            "PID": pid,
            "Class": class_value,
            "Auto Area": auto_area,
            "Manual Area": manual_area
        }, ignore_index=True)

# 根据 prompt_dir 过滤掉指定前缀的 PID
manual_prompt_id = ['Jake_105', 'GL_106', 'GL_118']
# 逻辑：根据 prompt_dir 中是否包含 "1" 或 "2" 来决定去除 manual_prompt_id 中的前几项
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
print("Remove prefixes:", remove_prefixes)
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]
df_results = filter_pid(df_results)


# 保存 CSV 文件
csv_save_path = os.path.join(data_dir, prompt_dir, prompt_dir + "_area_auto_vs_manual.csv")
df_results.to_csv(csv_save_path, index=False)
print(f"Results saved to {csv_save_path}")


0: Processing GL_104_L4-L5_sam2_seg_slice13_seg.nii.gz
1: Processing N_Melb_106_L4-L5_sam2_seg_slice17_seg.nii.gz
2: Processing Jake_103_L4-L5_sam2_seg_slice15_seg.nii.gz
3: Processing GL_106_L4-L5_sam2_seg_slice16_seg.nii.gz
4: Processing GL_117_L4-L5_sam2_seg_slice16_seg.nii.gz
5: Processing GL_108_L4-L5_sam2_seg_slice16_seg.nii.gz


/tmp/ipykernel_16780/4267081211.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = df_results._append({


6: Processing GL_112_L4-L5_sam2_seg_slice17_seg.nii.gz
7: Processing West_Bd_106_L4-L5_sam2_seg_slice11_seg.nii.gz
8: Processing GL_111_L4-L5_sam2_seg_slice16_seg.nii.gz
9: Processing GL_105_L4-L5_sam2_seg_slice14_seg.nii.gz
10: Processing GL_103_L4-L5_sam2_seg_slice17_seg.nii.gz
11: Processing GL_118_L4-L5_sam2_seg_slice18_seg.nii.gz
12: Processing GL_116_L4-L5_sam2_seg_slice16_seg.nii.gz
13: Processing Jake_105_L4-L5_sam2_seg_slice19_seg.nii.gz
14: Processing N_Melb_105_L4-L5_sam2_seg_slice17_seg.nii.gz
15: Processing West_Bd_115_L4-L5_sam2_seg_slice19_seg.nii.gz
16: Processing GL_114_L4-L5_sam2_seg_slice16_seg.nii.gz
17: Processing GL_109_L4-L5_sam2_seg_slice18_seg.nii.gz
18: Processing Jake_137_L4-L5_sam2_seg_slice17_seg.nii.gz
19: Processing GL_110_L4-L5_sam2_seg_slice22_seg.nii.gz
20: Processing Jake_138_L4-L5_sam2_seg_slice19_seg.nii.gz
21: Processing GL_107_L4-L5_sam2_seg_slice18_seg.nii.gz
22: Processing Jake_147_L4-L5_sam2_seg_slice12_seg.nii.gz
23: Processing GL_115_L4-L5_sa

In [5]:
print(df_results.shape)
df_results.head()



(176, 4)


PID Class  Auto Area  Manual Area
0  GL_104     1     11.256       11.192
1  GL_104     2     13.638       16.424
2  GL_104     3     13.739       15.775
3  GL_104     4      9.560        7.486
4  N_Melb     1     20.151       17.710

In [6]:
import numpy as np
from scipy.stats import ks_1samp, norm
from statsmodels.stats.weightstats import ttost_paired
from tqdm import tqdm

# 类别名称映射
class_names = {
    1: "Right_LES",
    2: "Right_MF",
    3: "Left_MF",
    4: "Left_LES"
}
# 自定义打印顺序
print_order = [4, 1, 3, 2]

# 参数
margin_pct = 0.05
n_bootstrap = 10000

# 用于记录每类的最小margin和对应p值
min_margins = []

for class_value in print_order:
    label_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    label = df_class["Manual Area"].values
    auto = df_class["Auto Area"].values
    min_len = min(len(label), len(auto))
    label, auto = label[:min_len], auto[:min_len]
    diff = auto - label

    # 正态性检验
    z = (diff - np.mean(diff)) / np.std(diff, ddof=1)
    ks_stat, ks_p = ks_1samp(z, norm.cdf)
    delta = np.mean(label) * margin_pct

    print(f"==== Class: {label_name} ====")
    print(f"Sample size: {min_len}")
    print(f"[KS normality] p = {ks_p:.4f}")

    if ks_p >= 0.05:
        # parametric TOST
        p_tost, *_ = ttost_paired(auto, label, -delta, delta)
        print("→ Using parametric TOST")
        print(f"pTOST = {p_tost:.4f} {'✅ Equivalent' if p_tost < 0.05 else '❌ Not equivalent'}")

        # 总是执行 margin 搜索
        for m in np.arange(0.01, 0.501, 0.001):
            d = np.mean(label) * m
            p_equiv, *_ = ttost_paired(auto, label, -d, d)
            if p_equiv < 0.05:
                min_margins.append((m * 100, p_equiv))
                print(f"→ Minimum margin for equivalence: δ = ±{m*100:.2f}% at p value {p_equiv:.3f}\n")
                break
        else:
            min_margins.append((np.nan, np.nan))
            print("→ Even δ = ±50% is not enough for equivalence\n")

    else:
        # bootstrap TOST
        inside = 0
        for _ in tqdm(range(n_bootstrap), leave=False):
            idx = np.random.choice(len(diff), len(diff), replace=True)
            mean_diff = np.mean(diff[idx])
            if -delta < mean_diff < delta:
                inside += 1
        p_boot = 1 - (inside / n_bootstrap)
        print("→ Using bootstrap TOST")
        print(f"bootstrap pTOST = {p_boot:.4f} {'✅ Equivalent' if p_boot < 0.05 else '❌ Not equivalent'}")

        # 总是执行 margin 搜索
        for m in np.arange(0.01, 0.501, 0.001):
            d = np.mean(label) * m
            inside_test = 0
            for _ in range(10000):
                idx = np.random.choice(len(diff), len(diff), replace=True)
                mean_diff = np.mean(diff[idx])
                if -d < mean_diff < d:
                    inside_test += 1
            p_equiv = 1 - (inside_test / 10000)
            if p_equiv < 0.05:
                min_margins.append((m * 100, p_equiv))
                print(f"→ Minimum margin for equivalence: δ = ±{m*100:.2f}%, at p value {p_equiv:.3f}\n")
                break
        else:
            min_margins.append((np.nan, np.nan))
            print("→ Even δ = ±50% is not enough for equivalence\n")

# 最后统一打印
print("### Minimum Margin Summary ###")
for margin, pval in min_margins:
    if not np.isnan(margin):
        print(f"{margin:.2f}% ({pval:.3f})")
    else:
        print("N/A (no equivalence within 50%)")


==== Class: Left_LES ====
Sample size: 44
[KS normality] p = 0.1164
→ Using parametric TOST
pTOST = 0.1001 ❌ Not equivalent
→ Minimum margin for equivalence: δ = ±6.40% at p value 0.048

==== Class: Right_LES ====
Sample size: 44
[KS normality] p = 0.3214
→ Using parametric TOST
pTOST = 0.3279 ❌ Not equivalent
→ Minimum margin for equivalence: δ = ±8.60% at p value 0.050

==== Class: Left_MF ====
Sample size: 44
[KS normality] p = 0.0235


→ Using bootstrap TOST
bootstrap pTOST = 0.2021 ❌ Not equivalent


→ Minimum margin for equivalence: δ = ±7.90%, at p value 0.046

==== Class: Right_MF ====
Sample size: 44
[KS normality] p = 0.6815
→ Using parametric TOST
pTOST = 0.7068 ❌ Not equivalent
→ Minimum margin for equivalence: δ = ±8.90% at p value 0.049

### Minimum Margin Summary ###
6.40% (0.048)
8.60% (0.050)
7.90% (0.046)
8.90% (0.049)


In [7]:
import pingouin as pg

# 初始化
epsilon = 1e-6
mape_results = []
icc_results = []

# 分析每一类
for class_value in print_order:
    label_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    label = df_class["Manual Area"].values
    auto = df_class["Auto Area"].values
    min_len = min(len(label), len(auto))
    label, auto = label[:min_len], auto[:min_len]

    # MAPE
    ape = np.abs((auto - label) / (label + epsilon))
    mape = np.mean(ape)
    mape_std = np.std(ape)
    mape_results.append([label_name, mape * 100, mape_std * 100])

    # ICC
    df_long = df_class.melt(id_vars=["PID"], value_vars=["Manual Area", "Auto Area"],
                            var_name="Method", value_name="Area")
    icc_df = pg.intraclass_corr(data=df_long, targets="PID", raters="Method", ratings="Area")
    icc_row = icc_df[icc_df["Type"] == "ICC2"].iloc[0]
    icc_value = icc_row["ICC"]
    ci_lower, ci_upper = icc_row.get("CI95%") if "CI95%" in icc_row else (np.nan, np.nan)
    icc_results.append([label_name, icc_value, ci_lower, ci_upper])

# 输出 MAPE
print("\n### MAPE Results ###")
for cls, m, s in mape_results:
    print(f"{cls}: {m:.1f} ± {s:.1f}%")

# 输出 ICC
print("\n### ICC Results ###")
for cls, icc, lo, hi in icc_results:
    print(f"{cls}: {icc:.3f}, [{lo:.2f} - {hi:.2f}]")



### MAPE Results ###
Left_LES: 18.7 ± 26.4%
Right_LES: 15.7 ± 18.6%
Left_MF: 12.4 ± 19.9%
Right_MF: 11.1 ± 8.3%

### ICC Results ###
Left_LES: 0.728, [0.53 - 0.85]
Right_LES: 0.795, [0.56 - 0.90]
Left_MF: 0.311, [-0.02 - 0.58]
Right_MF: 0.642, [0.33 - 0.82]


In [8]:
# 针对 Class 2，从原始数据中选取该类别
class2_df = df_results[df_results["Class"] == 2].copy()

# 查看 Class 2 中的异常值（假设已经检测到 PID "Jake_146" 是异常）
print("### Outliers for Class 2 ###")
class2_df["Abs_Diff"] = np.abs(class2_df["Manual Area"] - class2_df["Auto Area"])
print(class2_df[class2_df["PID"] == "Jake_146"][["PID", "Manual Area", "Auto Area", "Abs_Diff"]])

# 排除异常值 PID "Jake_146"
class2_no_outlier = class2_df[class2_df["PID"] != "Jake_146"]

# 构造长格式数据用于 ICC 计算
df_long_no_outlier = class2_no_outlier.melt(id_vars=["PID"], 
                                            value_vars=["Manual Area", "Auto Area"],
                                            var_name="Method", 
                                            value_name="Area")

# 使用 pingouin 计算 ICC（ICC2: 双向随机效应模型）
icc_df_no_outlier = pg.intraclass_corr(data=df_long_no_outlier, 
                                       targets="PID", 
                                       raters="Method", 
                                       ratings="Area")

# 提取 ICC2 对应的结果
icc_row_no_outlier = icc_df_no_outlier[icc_df_no_outlier["Type"] == "ICC2"].iloc[0]
icc_value_no_outlier = icc_row_no_outlier["ICC"]
ci_lower_no_outlier, ci_upper_no_outlier = icc_row_no_outlier["CI95%"]

print(f"\nAfter removing Jake_146, ICC for Class 2 is: {icc_value_no_outlier:.3f}, [{ci_lower_no_outlier:.2f} - {ci_upper_no_outlier:.2f}]")


### Outliers for Class 2 ###
          PID  Manual Area  Auto Area  Abs_Diff
133  Jake_146       12.694     16.756     4.062

After removing Jake_146, ICC for Class 2 is: 0.681, [0.39 - 0.84]
